In [38]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',50)


In [39]:
!pip install pgeocode
import pgeocode
nomi = pgeocode.Nominatim('IN')
import warnings
warnings.simplefilter('ignore')

You should consider upgrading via the 'c:\users\kuldeep\anaconda3\python.exe -m pip install --upgrade pip' command.


In [40]:
ls

 Volume in drive C is OS
 Volume Serial Number is 1455-5943

 Directory of C:\Users\Kuldeep\Capstone Project

23-08-2020  12:56    <DIR>          .
23-08-2020  12:56    <DIR>          ..
23-08-2020  12:40    <DIR>          .ipynb_checkpoints
23-08-2020  12:56            51,861 Data_Cleaning_And_Merging.ipynb
               1 File(s)         51,861 bytes
               3 Dir(s)  1,046,356,398,080 bytes free


In [41]:
invoice_data = pd.read_csv('E:\CodeChef\Capstone Project\Input_DataSet\Final_invoice.csv')
customer_data = pd.read_excel('E:\CodeChef\Capstone Project\Input_DataSet\Customer_Data.xlsx')

In [42]:
print("invoice data shape" , invoice_data.shape)
print("customer data shape" ,customer_data.shape)

invoice data shape (492314, 59)
customer data shape (555338, 9)


In [43]:
## dropping records more than 50% missing values from customer data 
customer_data = customer_data.drop(['Title','Marital Status','Occupation','Date of Birth','Death date'],axis=1)

# Imputing Partner type & Data origin with mode

customer_data['Partner Type'] = customer_data['Partner Type'].fillna(customer_data['Partner Type'].mode()[0])
customer_data['Data Origin'] = customer_data['Data Origin'].fillna(customer_data['Data Origin'].mode()[0])

In [44]:
## dropping records more than 50% missing values
invoice_data = invoice_data.drop(['Amt Rcvd From Custom','Amt Rcvd From Ins Co',
                                          'CGST(14%)','CGST(2.5%)', 'CGST(6%)','IGST(12%)','CGST(9%)',
                                          'IGST(18%)','IGST(28%)','IGST(5%)','SGST/UGST(14%)',
                                          'SGST/UGST(2.5%)','SGST/UGST(6%)','SGST/UGST(9%)',
                                          'TDS amount','Service Advisor Name','Outstanding Amt',
                                        'Total CGST','Total GST','Total IGST','Total SGST/UGST',
                                          'Policy no.','Cash /Cashless Type','Expiry Date','Gate Pass Date',
                                          'Insurance Company','Claim No.'
                                         ],axis=1)

### Deriving invoice date time , job card date time & service time

In [45]:
invoice_data["Invoice_DateTime"]=invoice_data["Invoice Date"]+' '+invoice_data["Invoice Time"]
invoice_data["JobCard_DateTime"]=invoice_data["JobCard Date"]+' '+invoice_data["JobCard Time"]
invoice_data['JobCard_DateTime'] = pd.to_datetime(invoice_data['JobCard_DateTime'],dayfirst=True)
invoice_data['Invoice_DateTime'] = pd.to_datetime(invoice_data['Invoice_DateTime'],dayfirst=True)
invoice_data["Service_Time"]=invoice_data["Invoice_DateTime"]-invoice_data["JobCard_DateTime"]

In [46]:
invoice_data.drop(['JobCard Date','JobCard Time','Invoice Date','Invoice Time'],axis = 1,inplace=True)

### Cleaning "Area/Locality", "City" and "District" columns using pgeocode library  

In [47]:
nomi.query_postal_code('400615')

postal_code                               400615
country code                                  IN
place_name        Kasarvadavali, Ghodbunder Road
state_name                           Maharashtra
state_code                                    16
county_name                                Thane
county_code                                  517
community_name                             Thane
community_code                               NaN
latitude                                 19.1941
longitude                                73.0002
accuracy                                       1
Name: 0, dtype: object

In [48]:
invoice_data["Clean_CITY"]=nomi.query_postal_code(np.array(invoice_data["Pin code"].astype(str))).county_name
invoice_data["Clean_District"]=nomi.query_postal_code(np.array(invoice_data["Pin code"].astype(str))).state_name
invoice_data["Clean_Location_Area"]=nomi.query_postal_code(np.array(invoice_data["Pin code"].astype(str))).place_name

### Dropping old "Area/Locality", "City" and "District" columns 

In [49]:
invoice_data.drop(["Area / Locality","CITY","District"],1,inplace=True)

### Storing Cleaned Invoice & Customer Data 

In [50]:
invoice_data.to_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\Invoice_Data.csv')
customer_data.to_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\Customer_Data.csv')

### Reading cleaned Invoice & Customer Data 

In [51]:
invoice_data = pd.read_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\Invoice_Data.csv',index_col= False)
customer_data = pd.read_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\Customer_Data.csv',index_col = False)
jtd_data = pd.read_csv('E:\CodeChef\Capstone Project\Input_DataSet\JTD.csv')

In [52]:
print("invoice data shape" , invoice_data.shape)
print(" customer data shape" ,customer_data.shape)
print(" jtd data shape", jtd_data.shape)

invoice data shape (492314, 32)
 customer data shape (555338, 5)
 jtd data shape (5619484, 10)


In [53]:
customer_data.isnull().sum()

Unnamed: 0          0
Business Partner    0
Customer No.        0
Partner Type        0
Data Origin         0
dtype: int64

In [54]:
invoice_data.isnull().sum()

Unnamed: 0                  0
Unnamed: 0.1                0
Cust Type                   0
Customer No.                0
Gate Pass Time              0
Invoice No                  0
Job Card No                 0
KMs Reading                 0
Labour Total                0
Make                        0
Misc Total                  0
Model                    1525
ODN No.                197286
OSL Total                   0
Order Type                  0
Parts Total                 0
Pin code                    0
Plant                       0
Plant Name1                 0
Print Status                0
Recovrbl Exp                0
Regn No                    16
Technician Name        236834
Total Amt Wtd Tax.          0
Total Value            197286
User ID                     0
Invoice_DateTime            0
JobCard_DateTime            0
Service_Time                0
Clean_CITY              47246
Clean_District          47246
Clean_Location_Area     47246
dtype: int64

In [55]:
jtd_data.isnull().sum()

Unnamed: 0                   0
DBM Order                    0
Order Item                   0
Material                 51629
Labor Value Number     3774033
Description              51629
Item Category                0
Order Quantity               0
Target quantity UoM      51629
Net value                    0
dtype: int64

#### Dropping unncessary columns & columns with more missing values

In [56]:
# # dropping unnecessary columns
customer_data.drop(['Unnamed: 0'],axis=1,inplace=True)
invoice_data.drop(['Unnamed: 0','ODN No.','Technician Name','Regn No','Total Value'],axis=1,inplace=True)
jtd_data.drop(['Unnamed: 0','Labor Value Number'],axis=1,inplace=True)

In [57]:
jtd_data.head()

,DBM Order,Order Item,Material,Description,Item Category,Order Quantity,Target quantity UoM,Net value
0,22385,180,OSLCC_SRV31,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87
1,23910,10,1DM279018170117,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13
2,23910,40,1IS150203BE01,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92
3,23910,60,1DM279001990173,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14
4,23910,70,1DM88552790S001SA,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63


In [58]:
invoice_data.head()

,Unnamed: 0.1,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,Clean_CITY,Clean_District,Clean_Location_Area
0,0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53.000000000,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S."
1,1,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14.000000000,Thane,Maharashtra,"Thane East, Kopri Colony"
2,2,Retail,81055,00:00:00,7005200004,173889,33985,180.19,MARUTI SUZUKI,0.00,ZEN,0.00,Running Repairs,52.95,400607,BC01,THANE,NO,0.0,233.14,BC01SA2,2015-01-03 11:12:57,2015-01-02 11:40:44,0 days 23:32:13.000000000,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh"
3,3,Retail,84419,00:00:00,7005200005,174158,78584,0.00,TATA MOTORS,0.00,INDICA,0.00,SMC Redemption,0.00,400603,BC01,THANE,NO,0.0,0.00,BC01SA2,2015-01-03 11:40:44,2015-01-03 10:12:32,0 days 01:28:12.000000000,Thane,Maharashtra,"Thane East, Kopri Colony"
4,4,Retail,18980,00:00:00,7005200006,173860,50057,1202.97,HYUNDAI,100.00,SANTRO XING,499.99,Paid Service,944.16,400603,BC01,THANE,NO,0.0,2747.12,BC01SA2,2015-01-03 12:07:17,2015-01-02 10:45:42,1 days 01:21:35.000000000,Thane,Maharashtra,"Thane East, Kopri Colony"


### Joining Invoice & Customer Data

In [59]:
inv_cust = pd.merge(invoice_data,customer_data,how='left',on='Customer No.')

In [60]:
inv_cust.shape

(492314, 30)

In [61]:
inv_cust.head()

,Unnamed: 0.1,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,Clean_CITY,Clean_District,Clean_Location_Area,Business Partner,Partner Type,Data Origin
0,0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53.000000000,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",NaN,NaN,NaN
1,1,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14.000000000,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN
2,2,Retail,81055,00:00:00,7005200004,173889,33985,180.19,MARUTI SUZUKI,0.00,ZEN,0.00,Running Repairs,52.95,400607,BC01,THANE,NO,0.0,233.14,BC01SA2,2015-01-03 11:12:57,2015-01-02 11:40:44,0 days 23:32:13.000000000,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh",NaN,NaN,NaN
3,3,Retail,84419,00:00:00,7005200005,174158,78584,0.00,TATA MOTORS,0.00,INDICA,0.00,SMC Redemption,0.00,400603,BC01,THANE,NO,0.0,0.00,BC01SA2,2015-01-03 11:40:44,2015-01-03 10:12:32,0 days 01:28:12.000000000,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN
4,4,Retail,18980,00:00:00,7005200006,173860,50057,1202.97,HYUNDAI,100.00,SANTRO XING,499.99,Paid Service,944.16,400603,BC01,THANE,NO,0.0,2747.12,BC01SA2,2015-01-03 12:07:17,2015-01-02 10:45:42,1 days 01:21:35.000000000,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN


### Joining inv_cust with jtd data

#### grouping the jtd data on dbm order to reduce duplicate order entries

In [62]:
jtd_group=jtd_data.groupby(["DBM Order","Item Category"],as_index=False).agg({"Net value":'sum',"Order Quantity":"sum"})

In [63]:
jtd_group.head()

,DBM Order,Item Category,Net value,Order Quantity
0,4,P001,3455.92,714.0
1,4,P002,11042.62,15.6
2,4,P010,2399.95,1.0
3,4,P011,100.00,1.0
4,5,P001,2310.47,283.0


In [64]:
jtd_group.shape

(1735310, 4)

In [65]:
invoice_customer_jtd=pd.merge(inv_cust,jtd_group,how="left",left_on="Job Card No",right_on="DBM Order")

In [66]:
invoice_customer_jtd.shape

(936275, 34)

In [67]:
invoice_customer_jtd.to_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\Cleaned_Merged_Data.csv')

In [68]:
invoice_customer_jtd.isnull().sum()

Unnamed: 0.1                0
Cust Type                   0
Customer No.                0
Gate Pass Time              0
Invoice No                  0
Job Card No                 0
KMs Reading                 0
Labour Total                0
Make                        0
Misc Total                  0
Model                    3185
OSL Total                   0
Order Type                  0
Parts Total                 0
Pin code                    0
Plant                       0
Plant Name1                 0
Print Status                0
Recovrbl Exp                0
Total Amt Wtd Tax.          0
User ID                     0
Invoice_DateTime            0
JobCard_DateTime            0
Service_Time                0
Clean_CITY              94162
Clean_District          94162
Clean_Location_Area     94162
Business Partner       894749
Partner Type           894749
Data Origin            894749
DBM Order                  39
Item Category              39
Net value                  39
Order Quan